# Black Scholes Exercise 5: Dask implementation

- Use cProfile and Line Profiler to look for bottlenecks and hotspots in the code, then use the diagnostic part of dask to look for help

In [1]:
#Boilerplate for the example

import cProfile
import pstats
import numpy as np
%load_ext line_profiler

import dask
import dask.array as da

from dask.array import log, sqrt, exp
from numpy import erf, invsqrt
# ------------------------------------
try:
    import numpy.random_intel as rnd
except:
    import numpy.random as rnd

# make xrange available in python 3
try:
    xrange
except NameError:
    xrange = range

SEED = 7777777
S0L = 10.0
S0H = 50.0
XL = 10.0
XH = 50.0
TL = 1.0
TH = 2.0
RISK_FREE = 0.1
VOLATILITY = 0.2
# RISK_FREE = np.float32(0.1)
# VOLATILITY = np.float32(0.2)
# C10 = np.float32(1.)
# C05 = np.float32(.5)
# C025 = np.float32(.25)
TEST_ARRAY_LENGTH = 1024

###############################################

def gen_data(nopt):
    return (
        rnd.uniform(S0L, S0H, nopt),
        rnd.uniform(XL, XH, nopt),
        rnd.uniform(TL, TH, nopt),
        )

nopt=100000
chunk=2000
price, strike, t = gen_data(nopt)
price = da.from_array(price, chunks=(chunk,), name=False)
strike = da.from_array(strike, chunks=(chunk,), name=False)
t = da.from_array(t, chunks=(chunk,), name=False)

# The Dask Black Scholes algorithm

In [2]:
def black_scholes ( nopt, price, strike, t, rate, vol):
    mr = -rate
    sig_sig_two = vol * vol * 2

    P = price
    S = strike
    T = t

    a = log(P / S)
    b = T * mr

    z = T * sig_sig_two
    c = 0.25 * z
    y = da.map_blocks(invsqrt, z)

    w1 = (a - b + c) * y
    w2 = (a - b - c) * y

    d1 = 0.5 + 0.5 * da.map_blocks(erf, w1)
    d2 = 0.5 + 0.5 * da.map_blocks(erf, w2)

    Se = exp(b) * S

    call = P * d1 - Se * d2
    put = call - P + Se
    
    return da.compute( da.stack((put, call)))



## Now run timeit, cProfile, and line_profiler

What do you notice in the profile info?

In [3]:
%timeit black_scholes(nopt, price, strike, t, RISK_FREE, VOLATILITY)

245 ms ± 20.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
cProfile.run('black_scholes(nopt, price, strike, t, RISK_FREE, VOLATILITY)')

         218136 function calls (215030 primitive calls) in 0.325 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:989(_handle_fromlist)
        1    0.000    0.000    0.326    0.326 <ipython-input-2-579bd37c6a4f>:1(black_scholes)
        1    0.000    0.000    0.326    0.326 <string>:1(<module>)
      582    0.000    0.000    0.000    0.000 _weakrefset.py:70(__contains__)
      395    0.001    0.000    0.001    0.000 abc.py:178(__instancecheck__)
        1    0.000    0.000    0.287    0.287 base.py:143(compute)
        2    0.000    0.000    0.000    0.000 base.py:185(<genexpr>)
        1    0.000    0.000    0.000    0.000 base.py:190(<listcomp>)
        2    0.000    0.000    0.000    0.000 base.py:198(<genexpr>)
        1    0.000    0.000    0.000    0.000 base.py:205(<listcomp>)
        2    0.000    0.000    0.001    0.001 base.py:209(<genexpr

        2    0.000    0.000    0.002    0.001 ufunc.py:69(__call__)
        2    0.000    0.000    0.000    0.000 ufunc.py:70(<listcomp>)
    103/1    0.000    0.000    0.000    0.000 utils.py:218(concrete)
       99    0.000    0.000    0.000    0.000 utils.py:384(dispatch)
    99/94    0.000    0.000    0.001    0.000 utils.py:410(__call__)
       47    0.000    0.000    0.000    0.000 utils.py:523(funcname)
        1    0.000    0.000    0.000    0.000 utils.py:794(ensure_dict)
        1    0.000    0.000    0.000    0.000 utils.py:808(package_of)
       26    0.000    0.000    0.000    0.000 {built-in method __new__ of type object at 0x10020a6b0}
        1    0.000    0.000    0.000    0.000 {built-in method _functools.reduce}
       26    0.000    0.000    0.000    0.000 {built-in method _hashlib.openssl_md5}
       54    0.000    0.000    0.000    0.000 {built-in method _operator.add}
       10    0.000    0.000    0.000    0.000 {built-in method _operator.mul}
        5    0.000

In [5]:
%lprun -f black_scholes black_scholes(nopt, price, strike, t, RISK_FREE, VOLATILITY)

## Now let's start the Local Cluster instead

Watch how the distributed mode changes things

In [6]:
from dask.distributed import Client
client = Client()

Go to http://localhost:8787/status

## Dask calling NumPy

Where are the NumPy differences?

In [7]:
from numpy import log, exp, erf, invsqrt

def black_scholes_numpy_mod ( nopt, price, strike, t, rate, vol ):
    mr = -rate
    sig_sig_two = vol * vol * 2

    P = price
    S = strike
    T = t

    a = log(P / S)
    b = T * mr

    z = T * sig_sig_two
    c = 0.25 * z
    y = invsqrt(z)

    w1 = (a - b + c) * y
    w2 = (a - b - c) * y

    d1 = 0.5 + 0.5 * erf(w1)
    d2 = 0.5 + 0.5 * erf(w2)

    Se = exp(b) * S

    call = P * d1 - Se * d2
    put = call - P + Se
    
    return np.stack((call, put))

def black_scholes_dask ( nopt, price, strike, t, rate, vol, schd=None ):
    return da.map_blocks( black_scholes_numpy_mod, nopt, price, strike, t, rate, vol, new_axis=0 ).compute(get = schd)


## Now run timeit, cProfile, and line_profiler

In [8]:
%timeit black_scholes_dask(nopt, price, strike, t, RISK_FREE, VOLATILITY)

1.07 s ± 39.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
cProfile.run('black_scholes(nopt, price, strike, t, RISK_FREE, VOLATILITY)')

         192023 function calls (166681 primitive calls) in 1.745 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     1205    0.001    0.000    0.005    0.000 <frozen importlib._bootstrap>:255(_module_repr)
     1125    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:410(has_location)
     1133    0.001    0.000    0.003    0.000 <frozen importlib._bootstrap>:570(_module_repr_from_spec)
       54    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:689(module_repr)
        8    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:762(module_repr)
        3    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:989(_handle_fromlist)
        1    0.001    0.001    1.745    1.745 <ipython-input-2-579bd37c6a4f>:1(black_scholes)
        1    0.000    0.000    1.746    1.746 <string>:1(<module>)
      100    0.000    0.000    0.000    0.000 __init__.py:1284(debug)
        2    0.00

        6    0.000    0.000    0.000    0.000 inspect.py:73(isclass)
        3    0.000    0.000    0.000    0.000 inspect.py:81(ismethod)
        3    0.000    0.000    0.000    0.000 inspect.py:91(ismethoddescriptor)
      102    0.000    0.000    0.002    0.000 ioloop.py:933(add_callback)
        7    0.000    0.000    0.000    0.000 itertoolz.py:241(unique)
       54    0.000    0.000    0.000    0.000 itertoolz.py:31(accumulate)
      101    0.000    0.000    0.000    0.000 itertoolz.py:362(first)
      105    0.000    0.000    0.000    0.000 itertoolz.py:468(concat)
       51    0.000    0.000    0.001    0.000 itertoolz.py:66(groupby)
       48    0.000    0.000    0.000    0.000 itertoolz.py:673(partition)
        7    0.000    0.000    0.000    0.000 itertoolz.py:727(count)
        7    0.000    0.000    0.000    0.000 itertoolz.py:739(<genexpr>)
       50    0.000    0.000    0.000    0.000 itertoolz.py:742(pluck)
       75    0.000    0.000    0.000    0.000 itertoolz.py:774

In [10]:
%lprun -f black_scholes_dask black_scholes_dask(nopt, price, strike, t, RISK_FREE, VOLATILITY)